In [1]:

import sys
import os 
import torch
import pandas as pd 
import torchaudio
import IPython.display as ipd
import numpy as np
import random 
from os.path import join 
from torch.utils.data import DataLoader, Dataset
from transformers import Wav2Vec2FeatureExtractor, AutoTokenizer, AutoModel
from torch import nn

sys.path.append("./lib/AngularGrad")
from myoptims.cosangulargrad import cosangulargrad
from sklearn.metrics import f1_score
import pytorch_lightning as pl

os.environ["TOKENIZERS_PARALLELISM"] = "false"

/home/oggy/anaconda3/envs/TorchCuda11.3/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 0. Process dataset CSV

In [2]:
# data = pd.read_csv('./data/iemocap_2024/iemocap_new_2024.csv')
# data_train = data[data['num_session'].isin([1,2,3,4])]
# data_test  = data[~data['num_session'].isin([1,2,3,4])]

In [3]:
# data_train.to_csv('./data/iemocap_2024/iemocap_train_new_2024.csv', index=False)
# data_test.to_csv('./data/iemocap_2024/iemocap_test_new_2024.csv',index=False)

## 1. Config 

In [4]:
class CFG:
    audio_length=8
    resampling_rate=16000

    num_labels=4 
    learning_rate=1e-4 
    min_lr=1e-6

    pooling_mode='max' 
    final_dropout=0.2
    model_name='HubertClassify_NoneCLIP'
    
    feature_speech=768 
    embedding_size = 512

    train_batch_size=32
    train_num_workers=4
    val_batch_size=64
    val_num_workers=4
    
    csv_train_path = './data/iemocap_2024/iemocap_train_new_2024.csv'
    csv_valid_path = './data/iemocap_2024/iemocap_test_new_2024.csv'    
    
    speech_pretrained_path ='./pretrained/hubert-base-ls960'
    speech_clip_pretrained ='./Output/Hubert_DistilBet_CLIP_0.119_8s.pth' 



## 1. Dataset Loader 

In [5]:
def Convert2Mono(audio_tensor ,channel_first: bool =True):
    if len(audio_tensor.shape)==1:
        return audio_tensor.unsqueeze(0)
    else:
        output = torch.mean(audio_tensor, dim=0) if channel_first else torch.mean(audio_tensor, dim=1)
        return output.unsqueeze(0)


def truncate_audio(audio_tensor, 
                   sampling_rate: int, 
                   max_time: int):
    '''
    Resize to the same length (padding or truncate)
    Ref: https://tiensu.github.io/blog/70_audio_deep_learning_part_4/
    Thay đổi chiều dài của tất cả các files Audio về chung một giá trị max_length. 
    Audio < max_length: sẽ được kéo dài bằng cách thêm vào khoảng im lặng - slience. 
    Audio > max_length: cắt bớt đi.

    audio_array: [Tensor]:  
    sampling_rate [int] 
    max_time: [int]
    '''

    num_rows, audio_len = audio_tensor.shape # channel_num, frame_num
    max_len = sampling_rate*max_time
    
    # Cut audio 
    if (audio_len > max_len):
        audio_tensor = audio_tensor[:,:max_len]

    # Padding to add at the beginning and end of the signal
    elif (audio_len < max_len):
        pad_begin_len = random.randint(0, max_len - audio_len)
        pad_end_len = max_len - audio_len - pad_begin_len

        # Pad with 0-values 
        pad_begin = torch.zeros((num_rows, pad_begin_len))
        pad_end = torch.zeros((num_rows, pad_end_len))
        audio_tensor = torch.cat((pad_begin, audio_tensor, pad_end), 1)

    return audio_tensor


def get_audio_path(file_name):
    return join('./data/iemocap_2024/iemocap/',file_name+'.wav')


def get_class_num(class_name):
    if class_name == 'neu':
        return 0 
    elif class_name == 'sad':
        return 1
    elif class_name == 'ang':
        return 2
    elif class_name == 'exc' or class_name == 'hap':
        return 3 
    else:
        return -1 

In [6]:
class Hubert_Classification_Dataset(Dataset):
    def __init__(self, 
            csv_path: str, 
            audio_length: int = None,
            resampling_rate: int = None,
            speech_pretrained_path: str = None
    ) -> None:
        super().__init__()
        
        self.df = pd.read_csv(csv_path)
        self.df['audio_paths']  = self.df['FileName'].apply(get_audio_path)
        self.df['class_labels'] = self.df['Label'].apply(get_class_num) 

        self.audio_paths = list(self.df['audio_paths'])
        self.labels  = list(self.df['class_labels'])

        self.audio_length = audio_length
        self.resampling_rate = resampling_rate

        self.hubert_processor = Wav2Vec2FeatureExtractor.from_pretrained(
            speech_pretrained_path, 
            local_files_only=True
        )

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        # 0. Read the audio 
        speech, sampling_rate = torchaudio.load(self.audio_paths[idx])
        
        # 1. Resample Audio 
        resampler = torchaudio.transforms.Resample(sampling_rate, self.resampling_rate)
        speech = resampler(speech).squeeze()

        # 2. Convert Stero2Mono 
        speech = Convert2Mono(speech, channel_first=True) 
        
        # 3. Resize make sure all audio same length  
        speech = truncate_audio(
            audio_tensor=speech, 
            sampling_rate=self.resampling_rate, 
            max_time=self.audio_length
        )

        # 4. Wave2Vec process audio --> return audio array (channel, signal) [Tensor]
        speech = self.hubert_processor(
            raw_speech=speech,
            padding=False,
            max_length=None,
            truncation=False,
            return_tensors='pt',
            sampling_rate=self.resampling_rate,
        ).input_values[0]

        # 5. Label  
        d_type = torch.long if isinstance(self.labels[idx], int) else torch.float
        label = torch.tensor(self.labels[idx], dtype=d_type)
        
        return {'audio': speech, 'label': label}

## 2. Lightning Dataloader 

In [7]:
import logging
from pytorch_lightning import LightningDataModule, LightningModule

class Hubert_Text_Dataset(LightningDataModule):
    def __init__(self, csv_train_path, csv_valid_path):
        super().__init__()
        self.csv_train_path = csv_train_path
        self.csv_valid_path = csv_valid_path
        self.dataloader = Hubert_Classification_Dataset

    def prepare_data(self):
        pass 
    @property
    def num_classes(self) -> int:
        return 4 #int(self.train_csv['class_labels'].nunique())   
    
    def setup(self, *_, **__) -> None:
        self.train_dataset = self.dataloader(
            csv_path=CFG.csv_train_path, 
            audio_length=CFG.audio_length,
            resampling_rate=CFG.resampling_rate,
            speech_pretrained_path=CFG.speech_pretrained_path
        )
        logging.info(f"training dataset: {len(self.train_dataset)}")
        
        self.val_dataset = self.dataloader(
            csv_path=CFG.csv_valid_path, 
            audio_length=CFG.audio_length,
            resampling_rate=CFG.resampling_rate,
            speech_pretrained_path=CFG.speech_pretrained_path
        )
        logging.info(f"validation dataset: {len(self.val_dataset)}")


    def train_dataloader(self) -> DataLoader:
        return DataLoader(
                self.train_dataset,
                batch_size=CFG.train_batch_size ,
                num_workers=CFG.train_num_workers,
                pin_memory=True,
                drop_last=True, 
                shuffle=True
            )
    
    def val_dataloader(self) -> DataLoader:
        return DataLoader(
                self.val_dataset,
                batch_size=CFG.val_batch_size ,
                num_workers=CFG.val_num_workers,
                pin_memory=True,
                drop_last=False,
                shuffle=False
            )


### 4. Projection Head + Speech Encoder 

In [8]:
class ProjectionHead(nn.Module):
    def __init__(self, embedding_dim: int, projection_dim: int, dropout: float) -> None:
        super().__init__()

        self.projection = nn.Linear(embedding_dim, projection_dim)
        self.gelu = nn.GELU()
        self.fc = nn.Linear(projection_dim, projection_dim)
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(projection_dim)

    def forward(self, x):
        projected = self.projection(x)
        x = self.gelu(projected)
        x = self.fc(x)
        x = self.dropout(x)
        x += projected
        return self.layer_norm(x)

In [9]:
class HubertForSpeechEncoder(nn.Module):
    def __init__(self, pooling_mode, pretrained_path):
        super().__init__()
        
        self.pooling_mode = pooling_mode
        self.hubert = AutoModel.from_pretrained(pretrained_path)
        self.project = ProjectionHead(
            embedding_dim=CFG.feature_speech,
            projection_dim=CFG.embedding_size,
            dropout=0.2,
        )
    
    # 1. Freeze CNN encoder 
    def freeze_cnn_encoder(self):
        self.hubert.feature_extractor.requires_grad_(False)
    
    # 2. Freeze feature projection (hubert)  
    def freeze_feat_projection(self):
        self.hubert.feature_projection.requires_grad_(False)

    # 3. Freeze transformer block     
    def freeze_transformer(self):
        self.hubert.encoder.requires_grad_(False)
    
    # 4. Freeze CLIP Projection 
    def freeze_clip_projection(self):
        self.project.requires_grad_(False)


    def merged_strategy(self, hidden_states, mode="mean"):
        if mode == "mean":
            outputs = torch.mean(hidden_states, dim=1)
        elif mode == "sum":
            outputs = torch.sum(hidden_states, dim=1)
        elif mode == "max":
            outputs = torch.max(hidden_states, dim=1)[0]
        else:
            raise Exception(
                "The pooling method hasn't been defined! Your pooling mode must be one of these ['mean', 'sum', 'max']")
        return outputs

    def forward(
            self,
            input_values,
            attention_mask=None,
            output_attentions=None,
            output_hidden_states=None,
            return_dict=None,
    ):  
        input_values = input_values.squeeze(1)
        outputs = self.hubert(
            input_values,
            attention_mask=attention_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        hidden_states = outputs[0]
        hidden_states = self.merged_strategy(hidden_states, mode=self.pooling_mode)
        speech_embedding = self.project(hidden_states)
        return speech_embedding

### 6. Classifier Model 

In [10]:
class HubertForSpeechClassifier(nn.Module):
    def __init__(self, num_labels, speech_clip_pretrained):
        super().__init__()
        
        self.speech_model = HubertForSpeechEncoder(
            pooling_mode=CFG.pooling_mode, 
            pretrained_path=CFG.speech_pretrained_path
        )

        # Load Pretrained 
        if speech_clip_pretrained is not None:
            self.speech_model.load_state_dict(torch.load(speech_clip_pretrained))
            print('Load CLIP Pretrained Success !!')
        
        # Classifier Head        
        self.gelu = nn.GELU()
        self.dropout = nn.Dropout(0.2)
        self.dense1 = nn.Linear(512, 256, bias=False)
        self.dense2 = nn.Linear(256, num_labels, bias=False)


    def forward(self, speech):
        feature = self.speech_model(speech)
        x  = self.dropout(feature)
        x = self.dense1(feature)
        x = self.gelu(x)
        x = self.dense2(x) 
        return x 

### 5. Base Training Loop

In [11]:
import torch.nn.functional as F 

def get_score(y_truth, y_pred):
    F1_score = f1_score(y_truth, y_pred, average='macro')
    return {"F1":F1_score}

class BaseTraining(LightningModule):
    def __init__(self, model):
        super().__init__()
        self.num_classes = CFG.num_labels
        self.learning_rate = CFG.learning_rate
        self.loss = nn.CrossEntropyLoss()
        self.model = model

    def forward(self, audio):
        return self.model(audio)

    def compute_loss(self, y_pred, y):
        return self.loss(y_pred, y)

    def training_step(self, batch, batch_idx):
        audio, label = batch['audio'], batch['label']
        y_pred = self.forward(audio)
        loss = self.compute_loss(y_pred, label)
        self.log("loss", loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        audio, label = batch['audio'],batch['label']
        y_pred = self.forward(audio)
        loss = self.compute_loss(y_pred, label)

        return {"preds":torch.argmax(y_pred.softmax(1), dim=1).detach().to('cpu').numpy(), 
                "truth":label.clone().detach().tolist(),
                "loss": loss.to('cpu').item()}

    def validation_epoch_end(self, outputs):
        predict_list = [] 
        target_list = []
        loss_list = [] 
        
        for out in outputs:
            predict_list.append(out['preds'])
            target_list.extend(out['truth'])
            loss_list.append(out['loss'])
        
        predict_list = np.concatenate(predict_list)
        target_list  = np.array(target_list)
        score = get_score(list(target_list), list(predict_list))
        loss  = np.array(loss_list).mean()

        self.log("F1", score['F1'],sync_dist=True,on_epoch=True,prog_bar=True)
        self.log('Loss_Val',loss,sync_dist=True,on_epoch=True,prog_bar=True)

        
    def configure_optimizers(self):
        # optimizer = torch.optim.AdamW(self.model.parameters(), lr=self.learn_rate)
        optimizer = cosangulargrad(self.model.parameters(), lr=self.learning_rate)  
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
            optimizer, 
            self.trainer.max_epochs, 
            CFG.min_lr
        ) 
        return [optimizer] , [scheduler]


### 7. Training Stage 1

In [ ]:
DataModule = Hubert_Text_Dataset(
    csv_train_path='./data/iemocap_2024/iemocap_train_new_2024.csv',
    csv_valid_path='./data/iemocap_2024/iemocap_test_new_2024.csv'
)
DataModule.setup()

In [ ]:
backbone = HubertForSpeechClassifier(CFG.num_labels, None) # Load pretrained CLIP 
backbone.speech_model.freeze_cnn_encoder() 
backbone.speech_model.freeze_feat_projection() 
backbone.speech_model.freeze_transformer()
backbone.speech_model.freeze_clip_projection() 

model = BaseTraining(backbone)

In [ ]:
# Logging
logger_name = f'{CFG.model_name}_TextAudio_CLIP_Classifier' 
logger = pl.loggers.CSVLogger(save_dir='./Output/', name=logger_name)
print("---->",logger_name)

# Checkpoint saving
ckpt = pl.callbacks.ModelCheckpoint(
    monitor='F1',
    save_top_k=1,
    save_last=True,     
    save_weights_only=True,
    filename='{epoch:02d}-{F1:.3f}',
    mode='max'
)

# Trainer 
trainer = pl.Trainer(
    accelerator='gpu',
    move_metrics_to_cpu=False,
    gpus=[0],
    callbacks=[ckpt],
    logger=logger,
    max_epochs=10,
    precision=32, # 16bit autocast 
    accumulate_grad_batches=1,
    check_val_every_n_epoch=1,
    # resume_from_checkpoint=''
)

# Training
trainer.fit(model=model, datamodule=DataModule)

### 8. Training Stage 2 

In [12]:
DataModule = Hubert_Text_Dataset(
    csv_train_path='./data/iemocap_2024/iemocap_train_new_2024.csv',
    csv_valid_path='./data/iemocap_2024/iemocap_test_new_2024.csv'
)
DataModule.setup()

In [13]:
backbone = HubertForSpeechClassifier(CFG.num_labels, None) # Load pretrained CLIP 
backbone.speech_model.freeze_cnn_encoder() 
# backbone.speech_model.freeze_feat_projection() 
# backbone.speech_model.freeze_transformer()
backbone.speech_model.freeze_clip_projection() 

model = BaseTraining(backbone)
weight = torch.load('/home/oggy/Desktop/AI_Sound_2024/Output/HubertClassify_NoneCLIP_TextAudio_CLIP_Classifier/version_0/checkpoints/epoch=08-F1=0.381.ckpt')
model.load_state_dict(weight['state_dict'])

<All keys matched successfully>

In [14]:
# Logging
logger_name = f'{CFG.model_name}_TextAudio_CLIP_Classifier' 
logger = pl.loggers.CSVLogger(save_dir='./Output/', name=logger_name)
print("---->",logger_name)

# Checkpoint saving
ckpt = pl.callbacks.ModelCheckpoint(
    monitor='F1',
    save_top_k=1,
    save_last=True,     
    save_weights_only=True,
    filename='{epoch:02d}-{F1:.3f}',
    mode='max'
)

# Trainer 
trainer = pl.Trainer(
    accelerator='gpu',
    move_metrics_to_cpu=False,
    gpus=[0],
    callbacks=[ckpt],
    logger=logger,
    max_epochs=20,
    precision=32, # 16bit autocast 
    accumulate_grad_batches=1,
    check_val_every_n_epoch=1,
    # resume_from_checkpoint=''
)

# Training
trainer.fit(model=model, datamodule=DataModule)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/home/oggy/anaconda3/envs/TorchCuda11.3/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:469: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  rank_zero_deprecation(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                      | Params
----------------------------------------------------
0 | loss  | CrossEntropyLoss          | 0     
1 | model | HubertForSpeechClassifier | 95.2 M
----------------------------------------------------
90.3 M    Trainable params
4.9 M     Non-trainable params
95.2 M    Total params
380.645   Total estimated model params size (MB)


----> HubertClassify_NoneCLIP_TextAudio_CLIP_Classifier
Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

2024-02-25 15:40:05.685390: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-25 15:40:05.685438: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-25 15:40:05.685557: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-25 15:40:05.685604: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-25 15:40:05.685736: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory f

Epoch 0:   0%|          | 0/154 [00:00<?, ?it/s]                      

2024-02-25 15:40:09.787629: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-25 15:40:09.787677: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-25 15:40:09.788267: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-25 15:40:09.788318: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-25 15:40:09.788528: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory f

...

Epoch 19:  87%|████████▋ | 134/154 [02:00<00:17,  1.12it/s, loss=0.00179, v_num=1, F1=0.720, Loss_Val=1.450] 

2024-02-25 16:23:36.199237: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-25 16:23:36.199287: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-25 16:23:36.200300: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-25 16:23:36.202021: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-25 16:23:36.202073: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factor

Epoch 19: 100%|██████████| 154/154 [02:11<00:00,  1.18it/s, loss=0.00179, v_num=1, F1=0.727, Loss_Val=1.430]

/home/oggy/anaconda3/envs/TorchCuda11.3/lib/python3.9/site-packages/pytorch_lightning/callbacks/progress/base.py:207: UserWarning: The progress bar already tracks a metric with the name(s) 'loss' and `self.log('loss', ..., prog_bar=True)` will overwrite this value.  If this is undesired, change the name or override `get_metrics()` in the progress bar callback.
  rank_zero_warn(


Epoch 19: 100%|██████████| 154/154 [02:11<00:00,  1.17it/s, loss=0.00179, v_num=1, F1=0.727, Loss_Val=1.430]


## 8. Saving Model 

In [ ]:
# class BaseSaving(LightningModule):
#     def __init__(self):
#         super().__init__()
#         self.speech_model = HubertForSpeechEncoder(
#             pooling_mode=CFG.pooling_mode, 
#             pretrained_path='./pretrained/hubert-base-ls960/'
#         )
#         self.text_model =  TextEncoder(
#             './pretrained/distilbert-base-uncased', 
#             trainable=True
#         )

#     def save_speech_weight(self,PATH):
#         torch.save(self.speech_model.state_dict(), os.path.join(PATH, "Speech_CLIP.pth"))
#         print(f'----> Saved: {os.path.join(PATH, "Speech_CLIP.pth")}')

In [ ]:
# model = BaseSaving()
# weight = torch.load('./Output/HubertTextClassify_TextAudio_CLIP/version_8/checkpoints/epoch=30-F1=0.000.ckpt')
# model.load_state_dict(weight['state_dict'])
# model.save_speech_weight('./Output')